## Important to install torch==1.12.0 and then to restart.
If any problems occur, to do with libcuda, its most likely because torch version is higher than 1.12.0.

This was not done via requirements - I've been trying, but running out of time.

In [7]:
pip install torch==1.12.0

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import pandas as pd

In [ ]:
folder_path = 'output'
data = {'file_name' : [], 'text': []}

for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            text_data = file.read().replace('\n', '')
        
        data['file_name'].append(filename)
        data['text'].append(text_data)

text_data_df = pd.DataFrame(data)
text_data_df.to_csv('text_data.csv', index=False)

print(text_data_df.head(2))

In [ ]:
!pip3 install "google-cloud-aiplatform>=1.25" "shapely<2.0.0"

In [ ]:
# Import Vertex AI SDK
PROJECT_ID = !gcloud config get project
PROJECT_ID = PROJECT_ID.n
LOCATION = "europe-west2"
LOCATION_DEPLOY = "europe-west2" #Location to deploy GCP resources

import vertexai
from google.cloud import aiplatform

vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

In [ ]:
import json
import time

import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('text_data.csv')

In [ ]:
!mkdir -p datasets
!gsutil copy datasets/vector_search_dataset.json gs://gen-ai-{PROJECT_ID}-bucket/embeddings/vs_root/vector_search_dataset.json

## Generating API endpoints for yourself - This may take up to 30 minutes.

This is only necessary on first run.

In [ ]:
DIMENSIONS = 768
GS_URI = "gs://gen-ai-%s-bucket/embeddings/vs_root/" % PROJECT_ID

gen_ai_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name="Gen AI Index",
    contents_delta_uri=GS_URI,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=5,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=10,
    leaf_nodes_to_search_percent=80,
    description="Example Index for Gen AI Playpen",
    location=LOCATION_DEPLOY
)

In [ ]:
gen_ai_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name="Gen AI Index Endpoint",
    description="Example Index for Gen AI Playpen",
    public_endpoint_enabled=True,
    location=LOCATION_DEPLOY
)

In [ ]:
gen_ai_index_endpoint = gen_ai_index_endpoint.deploy_index(
    index=gen_ai_index, deployed_index_id="gen_ai_deployed_index",
    machine_type="e2-standard-16",
    min_replica_count=1,
    max_replica_count=1
)

gen_ai_index_endpoint.deployed_indexes

## Important
Please go to google cloud. Go to dashboard and find project ID.
Replace in the two: playpen-8d8611 -> your project ID
index_name="projects/playpen-8d8611/locations/europe-west2/indexes/2680908415680643072"
index_endpoint_name="projects/playpen-8d8611/locations/europe-west2/indexEndpoints/5891412000042385408"

Then search for vector search. There should be an index and an indexendpoint tab. Go to index. Get ID. Replace: 2680908415680643072 -> your ID
index_name="projects/playpen-8d8611/locations/europe-west2/indexes/2680908415680643072"

Go to indexEndpoint. Get ID.
Replace: 5891412000042385408 -> your ID
index_endpoint_name="projects/playpen-8d8611/locations/europe-west2/indexEndpoints/5891412000042385408"

In [ ]:
gen_ai_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
    index_endpoint_name="projects/playpen-8d8611/locations/europe-west2/indexEndpoints/5891412000042385408"
)

gen_ai_index = aiplatform.MatchingEngineIndex(
    index_name="projects/playpen-8d8611/locations/europe-west2/indexes/2680908415680643072"
)

In [ ]:
from vertexai.preview.language_models import ChatModel

""" 
between using the most stable model in the last 6 month: chat-bison@001 
and the latest, most capable model, there is a massive difference in 
terms of context awareness, length of answer, ability to follow instructions,
etc. How do we measure these using metrics? I'm working on it.

I lied, the latest model, whilst sometimes its answers can be amazing,
definitely not as good when using RAG, it ignores context too often,
until I fix that, we'll work with the stable version.

"""
class PaLMWrapper:
    def __init__(self):
        self.chat_model = ChatModel.from_pretrained("chat-bison@001")
        self.parameters = {
            "temperature": 0.3,
            "max_output_tokens": 256,
            "top_p": 0.95,
        }
    
    def generate_response(self, context, message):
        chat = self.chat_model.start_chat(context=context)
        response = chat.send_message(message, **self.parameters)
        return response.text

In [ ]:
pw = PaLMWrapper()

## Replace the endpoints with your project id, index id, and indexendpoint id.

In [ ]:
import pandas as pd

import vertexai
from vertexai.preview.language_models import TextEmbeddingModel
from google.cloud import aiplatform

class VertexAIVectorStore:
    def __init__(self):
        
        # god awful way of doing it, should be a config and passed through but oh well hacky hack
        self.gen_ai_index_endpoint = aiplatform.MatchingEngineIndexEndpoint(
            index_endpoint_name="projects/playpen-8d8611/locations/europe-west2/indexEndpoints/8762456762491076608"
        )

        self.gen_ai_index = aiplatform.MatchingEngineIndex(
            index_name="projects/playpen-8d8611/locations/europe-west2/indexes/3025433787174486016"
        )
        
        self.model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
        self.df = pd.read_csv('text_data_g_embedding.csv')
        
    def search(self, input, k=3):
        embedding_vec =  self.model.get_embeddings([input])[0].values #Send request to embedding model to generate the embedding vector

        #find neighbours using vector search
        neighbours = self.gen_ai_index_endpoint.find_neighbors(
            deployed_index_id="gen_ai_deployed_index",
            queries=[embedding_vec],
            num_neighbors=k,
        )[0]
        
        results = []
        for nb in neighbours:
            nb_id = int(nb.id)
            if nb_id < len(self.df):
                url = self.df.iloc[int(nb.id)]['url']
                text = self.df.iloc[int(nb.id)]['text']
                score = nb.distance
                results.append((url, text, score))
            else:
                results=[('', '', 0)]
        
        return results


In [ ]:
vai = VertexAIVectorStore()

In [ ]:
import time

class RAG:
    def __init__(self, vector_store, palm_wrapper, initial_system_prompt=True):
        self.vector_store = vector_store
        self.palm_wrapper = palm_wrapper
        # the comment from mathew regarding memory
        self.conversation_history = []
        
        if initial_system_prompt:            
            system_prompt = r"You are a professional assistant with extensive experience helping numerous small and medium businesses. You work for a large retail bank called Lloyds. Please assist the user answering questions with detailed responses, providing reasoning whenever prescriptive advice is given. Ensure your answers are elaborate and helpful."
            self.conversation_history.append((system_prompt, ""))
        
    
    def generate(self, query, is_user_query=True, k=3):
        vector_start_time = time.time()
        
        contexts=[]
        sources=[]
        scores=[]
        
        if is_user_query:
            # if user query, perform vector db search
            # retrieve contexts based on the query
            search_results = self.vector_store.search(query, k)
            contexts = [str(result[1]) for result in search_results]
            sources = [result[0] for result in search_results]
            scores = [result[2] for result in search_results]
            
            if sum(scores)/len(scores) < 0.65:
                for i in range(len(contexts)):
                    
                    contexts[i]='In your search you could not find relevant context within the dataset. Therefore, you are uncertain about your response. IMPORTANT: Remind the customer that you are built to answer Lloyds banking related questions only.'
                    sources[i]='Could not find relevant sources'
            
              
        vector_end_time = time.time()
        # combine both query and response from conversation history
        history_context = '\n'.join(['Q: ' + query + '\nA: ' + response for query, response in self.conversation_history[-k:]])
        
        # combine history context and current contexts
        combined_context = '\n'.join([history_context] + contexts)
        combined_context = combined_context[-20000:]
        
        lm_start_time = time.time()
        response = self.palm_wrapper.generate_response(combined_context, query)
        lm_end_time = time.time()
        # update conversation history with current interaction
        self.conversation_history.append((query, response))
        
        vector_search_time = vector_end_time - vector_start_time
        lm_inference_time = lm_end_time - lm_start_time
        
        return response, sources

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

class ChatUI:
    def __init__(self, rag_instance):
        self.rag_instance = rag_instance
        self.conversation = []
        self._setup_ui()
    
    def _setup_ui(self):
        self.input_box = widgets.Text(
            placeholder='Message LBG help and support...',
            description='Lloyds',
            layout={'width': '80%'}
        )
        
        self.send_button = widgets.Button(
            description='Send',
            button_style='info',
            layout={'width': '12%'}
        )
        self.output_area = widgets.Output(layout={'border': '1px solid black', 'width': '100%'})
        self.send_button.on_click(self._on_send_clicked)
        
        input_send_box = widgets.HBox([self.input_box, self.send_button])
        
        display(self.output_area, input_send_box)
        
    def _on_send_clicked(self, b):
        query = self.input_box.value
        self.conversation.append(f"You: {query}")
        response = self.rag_instance.generate(query)
        self.conversation.append(f'LBG AI: {response}')
        
        with self.output_area:
            clear_output(wait=True)
            print('\n'.join(self.conversation))
        
        # clear input box
        self.input_box.value = ''
        
rag = RAG(vai, pw)
chat_ui = ChatUI(rag)    